In [2]:
import statsmodels.stats.api as sms
effect_size = 0.5 
alpha = 0.05       
power = 0.8       
analysis = sms.TTestIndPower()
required_sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')

print(f"Required sample size per group for 80% power: {required_sample_size:.2f}")

Required sample size per group for 80% power: 63.77


In [3]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [4]:
data = pd.read_excel("Behavior_Analysis_for_university_ Students.xlsx")

In [5]:
data.head()

,"Semester( number like: 1st, 2nd...)","Age( Years: 18, 19....)",Height( inches like:1 feet),Weight:( in kg),Gender,Religion,Permanent Residence,Current Residence,Family Type,"Number of siblings (numbers: 1, 2...)",...,I found it difficult to relax,\n I felt down-hearted and blue,I was intolerant of anything that kept me from getting on with what I was doing,I felt I was close to panic,I was unable to become enthusiastic about anything,I felt I wasn’t worth much as a person,I felt that I was rather touchy,"I was aware of the action of my heart in the absence of physical exertion (e.g. sense of heart rate increase, heart missing a beat)",I felt scared without any good reason,I felt that life was meaningless
0,5th,24,5.6,54,Male,Islam,Urban,Rented House or Mess,Nuclear,3,...,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all
1,8th,21,5.1,46 kg,Female,Islam,Rural,Rented House or Mess,Nuclear,2,...,Applied to me to a considerable degree or a go...,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all,Did not apply to me at all
2,8th,22,5 feet,50 kg,Female,Islam,Rural,Own House,Nuclear,0,...,"Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time",Did not apply to me at all,Did not apply to me at all,"Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time"
3,7th,22,5 feet 2 inch,63,Female,Islam,Urban,Own House,Nuclear,2,...,Did not apply to me at all,"Applied to me to some degree, or some of the time",Did not apply to me at all,Applied to me very much or most of the time,"Applied to me to some degree, or some of the time",Applied to me very much or most of the time,Did not apply to me at all,"Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time",Applied to me very much or most of the time
4,5th,23,"5.5""",62kg,Male,Islam,Rural,Rented House or Mess,Nuclear,1,...,"Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time",Did not apply to me at all,"Applied to me to some degree, or some of the time","Applied to me to some degree, or some of the time",Applied to me to a considerable degree or a go...,"Applied to me to some degree, or some of the time"


In [6]:
data.isnull().sum()

Semester( number like: 1st, 2nd...)                                                                                                                                                                 0
Age( Years: 18, 19....)                                                                                                                                                                             0
Height( inches like:1 feet)                                                                                                                                                                         0
Weight:( in kg)                                                                                                                                                                                     0
Gender                                                                                                                                                                                              0
Religion  

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 53 columns):
 #   Column                                                                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                                                                           --------------  ----- 
 0   Semester( number like: 1st, 2nd...)                                                                                                                                                              351 non-null    object
 1   Age( Years: 18, 19....)                                                                                                                                                                          351 non-null    object
 2   Height

In [8]:
columns_to_drop = ['Semester( number like: 1st, 2nd...)', 
                   'Age( Years: 18, 19....)', 
                   'Height( inches like:1 feet)', 
                   'Weight:( in kg)', 
                   'Religion', 
                   'Permanent Residence', 
                   'Number of siblings (numbers: 1, 2...)', 
                   "Father's Occupation", 
                   "Mother's Occupation", 
                   'If yes what are that...']
data = data.drop(columns=columns_to_drop)


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 43 columns):
 #   Column                                                                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                                                                           --------------  ----- 
 0   Gender                                                                                                                                                                                           351 non-null    object
 1   Current Residence                                                                                                                                                                                351 non-null    object
 2   Family

In [10]:
data=data.rename(columns= {
    'Gender': 'Gender',
    'Current Residence': 'Current Residence',
    'Family Type': 'Family Type',
    'Do you think your family environment is friendly?': 'Family Environment',
    'Academic performance(CGPA like: 3.5.....)': 'Academic Performance',
    'Daily average study hour': 'Study Hours',
    'Daily social media uses hours': 'Social Media Hours',
    'Relationship Status': 'Relationship Status',
    'Smoking status': 'Smoking Status',
    'Do you perform religious practice regularly?': 'Religious Practice',
    'How much satisfied with your academic activities(i,e...Presentations, Assignments, Tutorials..)': 'Academic Satisfaction',
    'Do you do any extra curricular activities?': 'Extra Curricular',
    'Do you feel that in the aspect of career building your subject has adequate opportunity in our country?': 'Career Opportunity',
    'Do your think your subject- related job gets enough social value in our country': 'Job Social Value',
    'Have you ever thought about or attempted to kill yourself?': 'Suicide Thought',
    'How often have you thought about killing yourself in the past year?': 'Suicide Frequency',
    'Have you ever told someone that you were going to commit suicide, or that you might do it?': 'Told About Suicide',
    'How likely is it that you will attempt suicide someday?': 'Suicide Likelihood',
    'How SATISFIED/dissatisfied are you with your current sleep pattern?': 'Sleep Satisfaction',
    'To what extent do you consider your sleep problem to INTERFERE with your daily functioning (e.g. daytime fatigue, ability to function at work/daily chores, concentration, memory, mood, etc.).': 'Sleep Interference',
    'How NOTICEABLE to others do you think your sleeping problem is in terms of impairing the quality of your life?': 'Sleep Noticeability',
    'How WORRIED/distressed are you about your current sleep problem?': 'Sleep Worry',
    'I found it hard to wind down': 'Hard To Relax',
    'I was aware of the dryness of my mouth': 'Dry Mouth',
    'I couldn’t seem to experience any positive feelings at all': 'No Positive Feelings',
    'I experienced breathing difficulty (e.g. excessively rapid breathing, breathlessness in the absence of physical exertion)': 'Breathing Difficulty',
    'I found it difficult to work up the initiative to do things': 'Low Initiative',
    'I tended to over-react to situations': 'Over-reacting',
    'I experienced trembling (e.g. in the hands)': 'Trembling',
    'I felt that I was using a lot of nervous energy': 'Nervous Energy',
    'I was worried about situations in which I might panic and make a fool of myself': 'Panic Worry',
    'I felt that I had nothing to look forward to': 'Nothing To Expect',
    'I found myself getting agitated': 'Agitation',
    'I found it difficult to relax': 'Difficulty Relaxing',
    'I felt down-hearted and blue': 'Down-hearted',
    'I was intolerant of anything that kept me from getting on with what I was doing': 'Intolerant Distraction',
    'I felt I was close to panic': 'Close To Panic',
    'I was unable to become enthusiastic about anything': 'No Enthusiasm',
    'I felt I wasn’t worth much as a person': 'Low Self-worth',
    'I felt that I was rather touchy': 'Touchiness',
    'I was aware of the action of my heart in the absence of physical exertion (e.g. sense of heart rate increase, heart missing a beat)': 'Heart Awareness',
    'I felt scared without any good reason': 'Unexplained Fear',
    'I felt that life was meaningless': 'Life Meaningless'
},inplace=False)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 43 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                --------------  ----- 
 0   Gender                                                                                                                                351 non-null    object
 1   Current Residence                                                                                                                     351 non-null    object
 2   Family Type                                                                                                                           351 non-null    object
 3   Family Environment                                                              

In [12]:
print(data.columns)

Index(['Gender', 'Current Residence', 'Family Type', 'Family Environment',
       'Academic Performance', 'Study Hours', 'Social Media Hours',
       'Relationship Status', 'Smoking Status', 'Religious Practice',
       'Academic Satisfaction', 'Extra Curricular', 'Career Opportunity',
       'Do your think your subject- related job gets enough social value in our country ',
       'Suicide Thought', 'Suicide Frequency', 'Told About Suicide',
       'Suicide Likelihood', 'Sleep Satisfaction', 'Sleep Interference',
       'Sleep Noticeability', 'Sleep Worry', 'Hard To Relax',
       '\n I was aware of the dryness of my mouth',
       '\nI couldn’t seem to experience any positive feelings at all',
       '\n I experienced breathing difficulty (e.g. excessively rapid breathing, breathlessness in the absence of physical exertion)',
       '\n I found it difficult to work up the initiative to do things',
       ' I tended to over-react to situations',
       '\nI experienced trembling (e.g.

In [13]:
remaining_columns = {
    'Do your think your subject- related job gets enough social value in our country ': 'Job Social Value',
    '\n I was aware of the dryness of my mouth': 'Dry Mouth',
    '\nI couldn’t seem to experience any positive feelings at all': 'No Positive Feelings',
    '\n I experienced breathing difficulty (e.g. excessively rapid breathing, breathlessness in the absence of physical exertion)': 'Breathing Difficulty',
    '\n I found it difficult to work up the initiative to do things': 'Low Initiative',
    ' I tended to over-react to situations': 'Over-reacting',
    '\nI experienced trembling (e.g. in the hands)': 'Trembling',
    '\nI felt that I was using a lot of nervous energy': 'Nervous Energy',
    ' I found it difficult to relax': 'Difficulty Relaxing',
    '\n I felt down-hearted and blue': 'Down-hearted',
    ' I was unable to become enthusiastic about anything': 'No Enthusiasm',
    ' I felt I wasn’t worth much as a person': 'Low Self-worth',
    ' I felt that I was rather touchy': 'Touchiness',
    ' I was aware of the action of my heart in the absence of physical exertion (e.g. sense of heart rate increase, heart missing a beat)': 'Heart Awareness',
    ' I felt scared without any good reason': 'Unexplained Fear'
}

data = data.rename(columns=remaining_columns)

In [14]:
print(data.columns)

Index(['Gender', 'Current Residence', 'Family Type', 'Family Environment',
       'Academic Performance', 'Study Hours', 'Social Media Hours',
       'Relationship Status', 'Smoking Status', 'Religious Practice',
       'Academic Satisfaction', 'Extra Curricular', 'Career Opportunity',
       'Job Social Value', 'Suicide Thought', 'Suicide Frequency',
       'Told About Suicide', 'Suicide Likelihood', 'Sleep Satisfaction',
       'Sleep Interference', 'Sleep Noticeability', 'Sleep Worry',
       'Hard To Relax', 'Dry Mouth', 'No Positive Feelings',
       'Breathing Difficulty', 'Low Initiative', 'Over-reacting', 'Trembling',
       'Nervous Energy', 'Panic Worry', 'Nothing To Expect', 'Agitation',
       'Difficulty Relaxing', 'Down-hearted', 'Intolerant Distraction',
       'Close To Panic', 'No Enthusiasm', 'Low Self-worth', 'Touchiness',
       'Heart Awareness', 'Unexplained Fear', 'Life Meaningless'],
      dtype='object')


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Gender                  351 non-null    object
 1   Current Residence       351 non-null    object
 2   Family Type             351 non-null    object
 3   Family Environment      351 non-null    object
 4   Academic Performance    346 non-null    object
 5   Study Hours             351 non-null    object
 6   Social Media Hours      351 non-null    object
 7   Relationship Status     351 non-null    object
 8   Smoking Status          351 non-null    object
 9   Religious Practice      351 non-null    object
 10  Academic Satisfaction   351 non-null    object
 11  Extra Curricular        351 non-null    object
 12  Career Opportunity      351 non-null    object
 13  Job Social Value        351 non-null    object
 14  Suicide Thought         351 non-null    object
 15  Suicid

In [16]:
data['Gender'].replace('Male',1,inplace=True)
data['Gender'].replace('Female',0,inplace=True)

C:\Users\AbdullahMiraz\AppData\Local\Temp\ipykernel_6916\1713749707.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Gender'].replace('Male',1,inplace=True)
C:\Users\AbdullahMiraz\AppData\Local\Temp\ipykernel_6916\1713749707.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a cop